# Scrape Online Travel Agent (OTA) Property Prices

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [71]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize the OTA scrape class

In [164]:
import sys
sys.path.insert(1, '../lib')
import otaWebScraper as otaws
import datetime

if debug:
    import importlib
    otaws = importlib.reload(otaws)
    
clsScraper = otaws.OTAWebScraper()

All packages in ExtractLoadTransform loaded successfully!


## Initialize the set of OTA URL for Scraping

In [171]:
from datetime import date

path = "../data"
file = "properties.json"
start_date = date(2022,8,30)
end_date = date(2022,9,2)

scrape_crietia_dict = {"pageOffset":25,
                        "pageUpperLimit":150,
                        "startDate": start_date,
                        "endDate" : end_date,
             }

_, _ota_url_parameterized_list  = clsScraper.build_scrape_url_list(dirPath=path, fileName=file, **scrape_crietia_dict)

#print(_ota_url_parameterized_list)
print("Completed parameterizing urls with %d instances." % (len(_ota_url_parameterized_list)))

Loaded 1 properties to begin scraping OTA data.
Processing booking.com ...
Completed parameterizing urls with 63 instances.


In [74]:
path = "../data"
file = "properties.json"
property_dict = clsScraper.load_ota_list(dirPath=path, fileName=file)
_scrape_tags_df = clsScraper.get_scrape_html_tags(property_dict)
print(_scrape_tags_df)

        variable   tag                   code          ota
0  content_block   div            .d20f4628d0  booking.com
0  property_name  span  fcab3ed991 a23c043802  booking.com
0      room_type  span             df597226dd  booking.com
0      room_rate   div  fcab3ed991 bd73d13072  booking.com
0   review_score   div  b5cd09854e d10a6220b4  booking.com
0       location   div             a1fbd102d9  booking.com
0          Other   div             d22a7c133b  booking.com


In [177]:
import os
from datetime import date, datetime
import pandas as pd

path = "../data/prices"

_today = date.today()
_hour = datetime.now().hour
_minute = datetime.now().minute
if _minute < 30:
    _minute = 0
else:
    _minute = 30
_s3block = "../data/property/rates/"
''' folder is a concaternation of date hour and minute; where minute < 30 --> 0 and 30 otherwise'''
_s3object = _s3block+str(_today)+"-"+str(_hour)+"-"+str(_minute)
#print(_s3object)

if not os.path.exists(_s3object):
    os.makedirs(_s3object)
#fPrefix = "Price"
counter = 0
_ota_tags_df = pd.DataFrame()
for ota_dict in _ota_url_parameterized_list:
    _ota_tags_df = _scrape_tags_df.loc[_scrape_tags_df['ota']==ota_dict['ota']]
    counter += 1
    ''' file name is concaternation of ota name + location + checkin date + page offset and .csv file extension'''
    _fname = ota_dict['ota']+"."+\
            ota_dict['location']+"."+\
            str(ota_dict['checkin'])+"."+\
            str(ota_dict['page_offset']).zfill(3)+\
            ".csv"
    _fname=_fname.replace(" ",".")

    if ota_dict['ota'] == 'booking.com':
        saveTo = clsScraper._scrape_bookings_to_csv(ota_dict['url'],ota_dict['checkin'],_fname, _s3object)
    else:
        print("Define a scraper function for %s, no data saved" % ota_dict['ota'])
#    _scraped_data_df = clsScraper.scrape_data_to_csv(ota_dict['url'],_ota_tags_df,_fname, _s3object)
#    _scraped_data_df = clsScraper._scrape_data_to_csv(ota_dict['url'],_fname, _s3object)


[Error]Class <WebScraper> Function <scrape_data_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/WebScraping/notebooks/../lib/otaWebScraper.py", line 428, in _scrape_bookings_to_csv
    room_type = _list.find('span', class_='df597226dd').text
AttributeError: 'NoneType' object has no attribute 'text'

[Error]Class <WebScraper> Function <scrape_data_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/WebScraping/notebooks/../lib/otaWebScraper.py", line 428, in _scrape_bookings_to_csv
    room_type = _list.find('span', class_='df597226dd').text
AttributeError: 'NoneType' object has no attribute 'text'

[Error]Class <WebScraper> Function <scrape_data_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/WebScraping/notebooks/../lib/otaWebScraper.py", line 428, in _scrape_bookings_to_csv
    room_type = 

In [175]:
print(str(ota_dict['checkin']))

2022-08-30


In [136]:
    print(_scraped_data_df)
    if _scraped_data_df.shape[0] > 0:
        saveTo = _s3object+"/"+_fname
        _scraped_data_df.to_csv(saveTo)
    break

                            room_type review_score  \
0   Grand Deluxe King Bed Non-Smoking          8.0   
0   Grand Deluxe King Bed Non-Smoking          8.0   
0   Grand Deluxe King Bed Non-Smoking          8.0   
0   Grand Deluxe King Bed Non-Smoking          8.0   
0   Grand Deluxe King Bed Non-Smoking          8.0   
..                                ...          ...   
0   Two Queen Bed Suite - Non-Smoking          NaN   
0   Two Queen Bed Suite - Non-Smoking          NaN   
0   Two Queen Bed Suite - Non-Smoking          NaN   
0   Two Queen Bed Suite - Non-Smoking          NaN   
0   Two Queen Bed Suite - Non-Smoking          NaN   

                                             location  \
0   Downtown Las Vegas - Fremont Street, Las Vegas...   
0   Downtown Las Vegas - Fremont Street, Las Vegas...   
0   Downtown Las Vegas - Fremont Street, Las Vegas...   
0   Downtown Las Vegas - Fremont Street, Las Vegas...   
0   Downtown Las Vegas - Fremont Street, Las Vegas...   
..       

SyntaxError: 'break' outside loop (3988095258.py, line 5)

# DEPRECATED

In [65]:
path = "../data"
#file = "property_urls.csv"
file = "properties.json"

clsScraper = otaws.OTAWebScraper()
#property_dict = clsScraper.load_ota_list(dirPath=path, fileName=file)

In [52]:
_ota_input_param_list, _ota_scrape_params = clsScraper.get_scrape_params(property_dict)
print(_ota_input_param_list)
print(_ota_scrape_param_df)

[{'ota': 'hotels.com', 'page': None, 'checkin': None, 'checkout': None, 'location': None, 'somekey': None}, {'ota': 'hotels.com', 'page': None, 'checkin': None, 'checkout': None, 'location': None, 'somekey': None}]
        source       param   tag                  class
0  booking.com   room_rate   div            .d20f4628d0
1  booking.com   room_type  span            .d20f4628d0
2  booking.com  hotel_name  span  fcab3ed991 a23c043802
3   hotels.com   room_rate   div            .d20f4628d0
4   hotels.com   room_type  span            .d20f4628d0
5   hotels.com  hotel_name  span  fcab3ed991 a23c043802


In [140]:
import pandas as pd

param_dict = {}
tag_dict = {}
_l_tag=[]
for prop_detail in property_dict:
    for detail in property_dict[prop_detail]:
        url = detail['url']
        for param in detail['inputs']:
            param_dict[param] = None
        for tag in detail['outputs']:
            for rate in tag['room_rate']:
                for key,value in rate.items():
                    tag_dict = {"source":prop_detail,
                                "param" : "room_rate",
                                "tag" : key,
                                "class" : value
                               }
                    _l_tag.append(tag_dict)
            for rate in tag['room_type']:
                for key,value in rate.items():
                    tag_dict = {"source":prop_detail,
                                "param" : "room_type",
                                "tag" : key,
                                "class" : value
                               }
                    _l_tag.append(tag_dict)
            for rate in tag['hotel_name']:
                for key,value in rate.items():
                    tag_dict = {"source":prop_detail,
                                "param" : "hotel_name",
                                "tag" : key,
                                "class" : value
                               }
                    _l_tag.append(tag_dict)

tag_df = pd.DataFrame(_l_tag,columns=["source","param","tag","class"])
print(tag_df)

        source       param   tag                  class
0  booking.com   room_rate   div            .d20f4628d0
1  booking.com   room_type  span            .d20f4628d0
2  booking.com  hotel_name  span  fcab3ed991 a23c043802


In [106]:
param_dict = {"page":"25",
              "checkin":"2022-08-30",
              "checkout":"2022-09-2",
              "location":"Las Vegas",
             }
for url in _l_urls:
    new_url = clsScraper.insert_params_in_url(url[0],**param_dict)
    print(new_url)

b
